In [1]:
import yaml
from story_sage.story_sage import StorySage
import os
import logging
import json
from story_sage.utils.local_entity_extractor import StorySageEntityExtractor
from openai import OpenAI
from story_sage.data_classes.story_sage_config import StorySageConfig
import httpx
from pydantic import BaseModel


CONFIG_PATH = './config.yml'

logger = logging.getLogger()

# Load configuration and data files
with open(CONFIG_PATH, 'r') as file:
    config = yaml.safe_load(file)

STORY_SAGE_CONFIG = StorySageConfig.from_config(config)


api_key = STORY_SAGE_CONFIG.openai_api_key
chroma_path = STORY_SAGE_CONFIG.chroma_path
chroma_collection = STORY_SAGE_CONFIG.chroma_collection

In [2]:
from story_sage.utils.embedding import update_tagged_entities, Embedder
import chromadb

chroma_client = chromadb.PersistentClient(path=config['CHROMA_PATH'])
embedder = Embedder()
vector_store = chroma_client.get_collection(config['CHROMA_COLLECTION'], embedding_function=embedder)

In [33]:
#chroma_client.delete_collection(config['CHROMA_COLLECTION'])

In [3]:
vector_store.peek(1)

{'ids': ['wheel_of_time_2_26_0'],
 'embeddings': array([[-3.20023969e-02, -2.68889945e-02, -5.15271351e-02,
         -2.08699182e-02,  9.89585891e-02, -2.13484862e-03,
          1.04317598e-01, -9.02495980e-02,  5.33699058e-02,
         -5.12670018e-02, -3.11007705e-02, -4.85098548e-03,
         -1.56757589e-02, -1.06297866e-01,  3.06304488e-02,
          1.72302767e-03, -5.16428798e-02,  2.83595435e-02,
         -3.03899329e-02,  3.71374027e-03, -1.03780791e-01,
         -9.59964097e-03,  1.14684545e-01,  5.56932203e-02,
          1.44887250e-02, -8.22873935e-02,  6.56040311e-02,
          8.51377472e-02,  1.45912906e-02,  1.81945416e-05,
          9.07592699e-02,  5.27487248e-02, -9.20181423e-02,
         -4.26945463e-02,  1.23610059e-02,  1.27470896e-01,
          6.92945300e-03, -9.93279442e-02, -1.56069342e-02,
         -3.85645358e-03,  2.60582585e-02, -2.18675807e-02,
         -2.35645231e-02,  4.11481559e-02, -2.14498416e-02,
         -1.11651339e-01, -3.09385769e-02, -6.110842

In [30]:
vector_store.get(ids=['wheel_of_time_1_5_3'], include=['metadatas', 'documents'])

{'ids': ['wheel_of_time_1_5_3'],
 'embeddings': None,
 'documents': ["Rand silently stalks through the forest, trying not to be heard or seen, gripped with fear from the strange creatures that attacked their farm. When Tam, his father, catches him, Rand feels relief. Tam warns him about the Trollocs—creatures that hunt men for pleasure and serve the Dark One. Despite Tam's injury, Rand decides to return to the farmhouse for supplies, feeling the need to protect his father. He stealthily approaches the farm, confused by shadows and the dark silence. As he crawls towards the sheep pen, he observes that the house is lit but remains wary of possible hidden dangers. He finally resolves to check the house first since he can see it clearly, unlike the dark barn."],
 'uris': None,
 'data': None,
 'metadatas': [{'book_number': 1,
   'chapter_number': 5,
   'characters': '[{"Rand": "Creeps through the forest, prepares to return to the farmhouse for supplies."}, {"Tam": "Catches Rand, warns him a

In [136]:
from uuid import uuid4

class CharacterSummary:
    
    def __init__(self, character_name: str, character_aliases: list[str] = None, character_id: str = None, chunk_summaries: dict[int, dict[int, list[str]]] = None):
        self.character_id = character_id or str(uuid4())
        self.character_name: str = character_name
        self.character_aliases: list[str] = character_aliases or [ character_name ]
        self.chunk_summaries: dict[int, dict[int, list[str]]] = chunk_summaries or {}

    
    
class CharacterCollection:

    def __init__(self):
        self.characters: dict[str, CharacterSummary] = {}

    def add_character(self, character_name: str, character_aliases: list[str] = None) -> CharacterSummary:
        new_character = CharacterSummary(character_name)
        self.characters[new_character.character_id] = new_character
        if character_aliases is not None:
            for alias in character_aliases:
                self.add_alias_to_character(character_name, alias)
        return new_character

    def add_summary_to_character(self, book_id: int, chapter_id: int, character_summary: dict[str, str]):
        target_character = self.find_character(character_summary['character_name'])
        if target_character is None:
            self.add_character(character_summary['character_name'])
            target_character = self.find_character(character_summary['character_name'])
        if book_id not in target_character.chunk_summaries:
            target_character.chunk_summaries[book_id] = {}
        if chapter_id not in target_character.chunk_summaries[book_id]:
            target_character.chunk_summaries[book_id][chapter_id] = []
        target_character.chunk_summaries[book_id][chapter_id].append(character_summary['summary'])

    def add_alias_to_character(self, character_name: str, character_alias: str):
        target_character = self.find_character(character_name)
        if target_character is None:
            self.add_character(character_name)
            target_character = self.find_character(character_name)
        target_character.character_aliases.append(character_alias)

    def find_character(self, character_name: str):
        for _, character in self.characters.items():
            if str.lower(character.character_name) == str.lower(character_name):
                return character
            if str.lower(character_name) in [str.lower(alias) for alias in character.character_aliases]:
                return character
        return None
    
    def to_json(self):
        return json.dumps(self, default=lambda o: o.__dict__, sort_keys=True, indent=4)
    
    @staticmethod
    def from_json(json_string: str):
        dict_obj = json.loads(json_string)
        new_collection = CharacterCollection()
        for character_id, character_dict in dict_obj['characters'].items():
            new_collection.characters[character_id] = CharacterSummary(**character_dict)
        return new_collection


In [141]:
def get_character_set(chapter_number: int, character_collection: CharacterCollection):
    where_document = {
        '$and': [
            {'series_id': 3},
            {'book_number': 2},
            {'chapter_number': chapter_number}
        ]}

    docs = vector_store.get(where=where_document, include=['metadatas'])
    character_set = set()
    for metadata in docs['metadatas']:
        for charsum in json.loads(metadata['characters']):
            for character, summary in charsum.items():
                character = character.replace('’', "'")
                if not character_collection.find_character(character):
                    print(f'{character}: {summary}')

In [125]:
collection = CharacterCollection()
collection.add_character('Rand al\'Thor', character_aliases=['Rand'])
collection.add_character('Bran al\'Vere', character_aliases=['Bran', 'Master al\'Vere', 'Mayor Bran', "Bran al'Vere (the Mayor)", "Mayor al'Vere", 'Mayor'])
collection.add_character('Mistress al\'Vere', character_aliases=['Mistress al\'Vere'])
collection.add_character("Kari al'Thor", character_aliases=[ "Kari al'Thor (Rand's mother)" ])
collection.add_character('Egwene', character_aliases=['Egwene al\'Vere'])
collection.add_character('Cenn Buie', character_aliases=['Cenn'])
collection.add_character('Mat Cauthon', character_aliases=[ 'Mat', 'Matrim (Mat) Cauthon' ])
collection.add_character('Tam al\'Thor', character_aliases=[ 'Tam', 'Rand\'s father', 'Tam al\'Vere' ])
collection.add_character('Haral Luhhan', character_aliases=[ 'Haral', 'Master Luhhan', 'Luhhan', 'Haral Luhhan (Master Luhhan)' ])
collection.add_character('Lan', character_aliases=[ 'Lan Mandragoran' ])
collection.add_character('Moiraine Damodred', character_aliases=[ 'Moiraine', "Moiraine Sedai" ])
collection.add_character('Nynaeve al\'Meara', character_aliases=[ 'Nynaeve' ])
collection.add_character('Ewin Finngar', character_aliases=[ 'Ewin' ])
collection.add_character('Padan Fain', character_aliases=[ 'Fain', 'Mordeth' ])
collection.add_character('Perrin Aybara', character_aliases=[ 'Perrin' ])
collection.add_character('Thom Merrilin', character_aliases=[ 'Thom', 'The Gleeman' ])
collection.add_character('Hari Coplin', character_aliases=[ 'Hari' ])
collection.add_character("Myrddraal", character_aliases=[ 'Fade', 'Halfman' ])
collection.add_character("Ba'alzamon")
collection.add_character('Min', character_aliases=[ 'Min Farshaw' ])
collection.add_character('Dain Bornhald', character_aliases=[ 'Children of the Light (specifically Lord Bornhald)', 'Lord Bornhald' ])
collection.add_character('Geofram Bornhald')
collection.add_character('Bayle Domon', character_aliases=[ 'Domon', 'Captain Domon' ])
collection.add_character('Floran Gelb')
collection.add_character('Elyas Machera', character_aliases=[ 'Elyas' ])
collection.add_character('Hopper')
collection.add_character('Bela')
collection.add_character('Aram')
collection.add_character('Byar', character_aliases=[ 'Child Byar' ])
collection.add_character('Else Grinwell', character_aliases=[ 'Else' ])
collection.add_character('Howal Gode', character_aliases=[ 'The wealthy man (Howal Gode)', 'Gode' ])
collection.add_character('Logain Ablar', character_aliases=[ 'Logain' ])
collection.add_character('Taringail Damodred', character_aliases=[ 'Taringail' ])
collection.add_character('Queen Morgase', character_aliases=[ 'Morgase' ])
collection.add_character('Basel Gill', character_aliases=[ 'Gill', 'Master Gill' ])
collection.add_character('Gareth Bryne', character_aliases=[ 'Bryne', 'Lord Bryne' ])
collection.add_character('Loial')
collection.add_character('Elayne', character_aliases=[ 'Elayne Trakand' ])
collection.add_character('Gawyn', character_aliases=[ 'Gawyn Trakand' ])
collection.add_character('Galad', character_aliases=[ 'Galad Damodred' ])
collection.add_character('Tallanvor')
collection.add_character('Elaida', character_aliases=[ 'Elaida do Avriny a\'Roihan' ])
collection.add_character('Lord Agelmar', character_aliases=[ 'Agelmar' ])
collection.add_character('Ingtar Shinowa', character_aliases=[ 'Ingtar' ])
collection.add_character('Green Man', character_aliases=[ 'The Green Man' ])
collection.add_character('Aginor')
collection.add_character('Balthamel')

In [142]:
collection.add_character('ignore', character_aliases=[ 'He', 'Unnamed man' ])
c = 0
for n in range(c, c+5):
    get_character_set(n, collection)

Bors: Observes silently and shifts his cloak, concealing his frame.
Bors: Studies the guests and contemplates accepting wine; observes the dynamics and weaknesses of the attendees; ultimately submits to the Myrddraal.
Golden-haired servants: Circulate among guests, offering wine with wordless smiles.
Aes Sedai: Present in the gathering, identified by the serpent rings.
Trollocs: Enter the room, intimidating the guests by bowing to the Myrddraal.
Bors: Serves a master, wrestles with panic and curiosity, reclines and rises under Ba’alzamon's command.
Bors: Receives commands from Ba’alzamon, experiences overwhelming visions, and is escorted by a servant.
Servant: Leads Bors to his room and informs him about his departure.
Two men: dance atop a tower in Fal Dara.
Servants: Dart about preparing rooms and carrying orders.
Dark-eyed warriors: Walk with haste, excited about the Amyrlin's visit.
Elansu: Directs the women in gathering Rand's old clothes, emphasizing the instructions from Moirain

In [119]:
def get_character_summaries(vector_store, book_number, character_collection: CharacterCollection):

    target_series = next(series for series in STORY_SAGE_CONFIG.series if series.series_id == 3)
    num_chapters = target_series.books[book_number-1].number_of_chapters
    for chapter_number in range(1, num_chapters+1):
        where_document = {
            '$and': [
                {'series_id': 3},
                {'book_number': book_number},
                {'chapter_number': chapter_number}
            ]}

        docs = vector_store.get(where=where_document, include=['metadatas'], limit=100)
        metadatas = docs['metadatas']
        metadatas = sorted(metadatas, key=lambda x: x['chunk_index'])
        for metadata in docs['metadatas']:
            for charsum in json.loads(metadata['characters']):
                for character, summary in charsum.items():
                    character = character.replace('’', "'")
                    if character_collection.find_character(character):
                        character_collection.add_summary_to_character(book_number, chapter_number, { 'character_name': character, 'summary': summary })

In [126]:
get_character_summaries(vector_store, 1, collection)

In [129]:
with open('./chunks/wheel_of_time/summaries/character_summaries.json', 'w') as f:
    f.write(collection.to_json())

In [137]:
with open('./chunks/wheel_of_time/summaries/character_summaries.json', 'r') as f:
    collection = CharacterCollection.from_json(f.read())

In [140]:
collection.find_character('Moiraine').chunk_summaries

{'1': {'2': ['Appears unexpectedly; captivates Rand with her presence and attire.',
   'Engages with the boys, offers tasks, and gives them coins.'],
  '4': ['Interacts with Thom, asserting her identity and subtly revealing her status.'],
  '7': ['Offers to help Tam, reveals the burden of being an Aes Sedai.',
   'Calmly reassures Rand and continues on with determination.'],
  '8': ['Ignore others and focuses on healing Tam.',
   "Expresses regret for being unprepared against the Dark One's forces, uses an angreal to heal Tam.",
   "Reveals the necessity of Rand's departure and offers reassurance about safety.",
   'Advises Rand to leave a note and emphasizes the need for secrecy and urgency.'],
  '9': ['Faces an angry crowd of villagers accusing her; remains calm.',
   'Displays her power with her staff to silence the crowd and tells the story of Manetheren to inspire remembrance.'],
  '10': ['Makes decisions about who can join the journey and emphasizes urgency.',
   'Encourages quic

In [15]:
vector_store.query(query_texts="what's the name of harry's owl?", include=['metadatas', 'documents'])

{'ids': [['harry_potter_3_1_0',
   'harry_potter_1_5_2',
   'harry_potter_4_3_2',
   'harry_potter_4_15_0',
   'harry_potter_3_22_3',
   'harry_potter_5_3_0',
   'harry_potter_4_5_1',
   'harry_potter_4_21_0',
   'harry_potter_3_3_3',
   'harry_potter_3_4_1']],
 'embeddings': None,
 'documents': [[' Owl Post Harry Potter was a highly unusual boy in many ways.',
   "Hagrid brings Harry to Gringotts, revealing Harry's fortune in gold coins. After gathering some money, they visit a high-security vault where Hagrid retrieves a mysterious brown package. They then encounter a spoiled boy at Madam Malkin’s Robes shop, who boasts about being from a wealthy family and expresses disdain for those from Muggle backgrounds. Hagrid explains Quidditch and the school Houses to Harry, emphasizing the dangers of Slytherin. They shop for Harry's school supplies, visit an Apothecary for potion ingredients, and Hagrid decides to buy Harry an owl instead of a toad for his birthday. They leave the Owl Empori

In [3]:
client = OpenAI(
    api_key=STORY_SAGE_CONFIG.openai_api_key, http_client=httpx.Client(verify=False)
)

class RelevantChunks(BaseModel):
    chunk_ids: list[str]
    secondary_query: str

def identify_relevant_chunks(input_text: str, chunk_summaries: dict[str, str]) -> RelevantChunks:
    """Analyzes the ids and summaries provided and returns the ids of chunks
    that are most relevant to the input text."""
    summaries = "\n".join([f"- {id}: {doc}" for id, doc in chunk_summaries.items()])
    chat_completion = client.beta.chat.completions.parse(
        messages=[
            {
                "role": "system",
                "content": f"""
                    You will receive a list of summaries of chunked passages from bookes along with their IDs.
                    Based on the provided summaries, please identify the IDs of the chunks that are most relevant to the input text.
                    The summaries are in the format:

                    - <chunk_id>: <summary>

                    If the answer to the input text is not likely to be present based on the summaries, 
                        please provide a secondary query to send to the vector store that would help in identifying more relevant chunks.
                    For example, if the input text is about someone with a relationship to someone else,
                        write a query that would help in identifying who the other people in the relationship are.

                    Input text: {input_text}
                """
            },
            {
                "role": "user",
                "content": summaries
            }
        ],
        model="gpt-4o-mini",
        response_format=RelevantChunks
    )
    return chat_completion.choices[0].message.parsed


In [4]:
res = client.beta.chat.completions.parse(
    messages=[
        {"role": "system", "content": "What is the input text?"},
        {"role": "user", "content": "Harry Potter is a wizard who attends Hogwarts School of Witchcraft and Wizardry."}
    ],
    model="gpt-4o-mini"
)

In [7]:
print(res.usage.total_tokens)

56


In [27]:
query_texts="who is harry?"

filter = {'$and': [
            {'series_id': 2}, 
            {'$or': [
                {'book_number': {'$lt': 1}}, {'$and': [{'book_number': 1}, {'chapter_number': {'$lt': 99}}]}]}]}

results = vector_store.query(query_texts=query_texts, where=filter, n_results=50)
ids_documents_dict = {id: doc for id, doc in zip(results['ids'][0], results['documents'][0])}

#results = identify_relevant_chunks(input_text=query_texts, chunk_summaries=ids_documents_dict)

print('Full list of returned context:')
print([key for key, _ in ids_documents_dict.items()])
print('Selected context:')
print(results)

Full list of returned context:
[]
Selected context:
{'ids': [[]], 'embeddings': None, 'documents': [[]], 'uris': None, 'data': None, 'metadatas': [[]], 'distances': [[]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}


In [48]:
refined_results = vector_store.get(ids=results.chunk_ids, include=['metadatas'])
print(refined_results)

{'ids': ['harry_potter_1_6_0', 'harry_potter_1_6_1', 'harry_potter_1_3_1'], 'embeddings': None, 'documents': None, 'uris': None, 'data': None, 'metadatas': [{'book_number': 1, 'chapter_number': 6, 'characters': '[{"Harry": "Spends time in his room, reads books, interacts with Hedwig, and asks Uncle Vernon for a lift to King\'s Cross."}, {"Dudley": "Avoids Harry, screams and runs away from him."}, {"Uncle Vernon": "Ignores Harry initially but grunts in response to Harry\'s request."}, {"Aunt Petunia": "Ignores Harry and stops vacuuming."}]', 'chunk_index': '0', 'creatures': '["owl (Hedwig)"]', 'full_chunk': ' The Journey from Platform Nine and Three-Quarters Harry’s last month with the Dursleys wasn’t fun. True, Dudley was now so scared of Harry he wouldn’t stay in the same room, while Aunt Petunia and Uncle Vernon didn’t shut Harry in his cupboard, force him to do anything, or shout at him — in fact, they didn’t speak to him at all. Half terrified, half furious, they acted as though an

In [4]:
story_sage = StorySage(
    api_key=api_key,
    chroma_path=chroma_path,
    chroma_collection_name=chroma_collection,
    entities_dict=entities,
    series_list=series_list,
    n_chunks=10  # Number of text chunks to process
)

2024-12-30 22:01:38,651 - story_sage.story_sage - DEBUG - Logger initialized from parent.


In [50]:
import logging

data = {
    'question': "what is a parseltongue?",
    'book_number': 1,
    'chapter_number': 99,
    'series_id': 2
}

logging.getLogger().setLevel(logging.ERROR)
result, context, request_id = story_sage.invoke(**data)

NameError: name 'story_sage' is not defined